## Importing libraries

In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
from transformers import DistilBertForSequenceClassification, TrainingArguments

C:\Users\13400f_RTX4060ti\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
NVIDIA GeForce RTX 4060 Ti


## Importing dataset

In [2]:
ds=pd.read_csv(r"D:\Text_Sentiment\datasets\final_dataset.csv")
print(ds.shape)
print(ds.columns)
print(ds['emotion'].value_counts())
ds.dropna(inplace=True)
print(ds.isnull().sum())
print(len(ds))

(106355, 2)
Index(['text', 'emotion'], dtype='object')
emotion
fun           10000
surprise      10000
enthusiasm    10000
anger         10000
happiness     10000
hate          10000
love          10000
relief        10000
sadness        9999
neutral        9998
empty          6358
Name: count, dtype: int64
text       0
emotion    0
dtype: int64
106355


## Creating an emotion -> integer mapping column called 'labels'

In [3]:
emotions=ds['emotion'].unique().tolist()
print(emotions)
mapping={label: i for i, label in enumerate(emotions)}
ds['labels'] = ds['emotion'].map(mapping)

['fun', 'surprise', 'neutral', 'enthusiasm', 'happiness', 'hate', 'sadness', 'empty', 'love', 'relief', 'anger']


In [4]:
print(len(ds['text']))
print(len(ds['emotion']))
print(len(ds['labels']))

106355
106355
106355


## Splitting dataset into train, test and validation sets

In [5]:
X=ds['text']
y=ds['labels']
X_train,X_temp,y_train,y_temp=train_test_split(X,y,test_size=0.2,stratify=y,random_state=1) #stratify based on target (y) for a homogenous and unbiased sets
X_val, X_test, y_val, y_test=train_test_split(X_temp,y_temp,test_size=0.5,stratify=y_temp,random_state=1) #splitting the remaining 0.3 into 50% test and 50% val
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")
print(f"Validation set size: {len(X_val)}")

Training set size: 85084
Testing set size: 10636
Validation set size: 10635


## Using DistilBERT's specific tokenizer to create a embeddings of the features (X)

In [6]:
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors='pt')
sample_text = X_train.iloc[0]
sample_encoding = tokenizer(sample_text, return_tensors='pt')
print(sample_text)
print(sample_encoding)
print(f"Decoded Text: {tokenizer.decode(sample_encoding['input_ids'][0])}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_labels = torch.tensor(y_train.tolist())
val_labels = torch.tensor(y_val.tolist())
test_labels = torch.tensor(y_test.tolist())

Faced a major today. My first time on the bike to office. whew
{'input_ids': tensor([[ 101, 4320, 1037, 2350, 2651, 1012, 2026, 2034, 2051, 2006, 1996, 7997,
         2000, 2436, 1012, 1059, 5369, 2860,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Decoded Text: [CLS] faced a major today. my first time on the bike to office. whew [SEP]


## Creating a custom PyTorch dataset class

In [7]:
class EmotionDataset(torch.utils.data.Dataset):
    """
    A custom PyTorch Dataset class to interface the tokenized inputs
    and integer labels for the Hugging Face Trainer.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)
train_dataset = EmotionDataset(train_encodings, train_labels) #Creating objects of EmotionDataset class for the train dataset
val_dataset = EmotionDataset(val_encodings, val_labels) #Creating objects of EmotionDataset class for the val dataset
test_dataset = EmotionDataset(test_encodings, test_labels) #Creating objects of EmotionDataset class for the test dataset

print("PyTorch Dataset objects created successfully.")

PyTorch Dataset objects created successfully.


## Initializing the model and training hyperparameters (training_args)

In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(emotions))
model.to(device)
training_args = TrainingArguments(
    output_dir='./emotion_results',  # Directory to save model checkpoints
    num_train_epochs=3,              # Number of times to run through the entire training set (Adjust if needed)
    per_device_train_batch_size=16,  # Training batch size (Adjust if you run out of GPU memory)
    per_device_eval_batch_size=64,   # Evaluation batch size (can be larger than training batch size)
    learning_rate=2e-5,
    warmup_steps=500,                # Learning rate scheduler warm-up steps
    weight_decay=0.01,               # L2 regularization
    logging_dir='./emotion_logs',    # Directory for logs
    logging_steps=100,               # Log training status every 100 steps
    save_strategy="epoch",     # Save best model version after each epoch
    eval_strategy="epoch",     # Run evaluation on the validation set after every epoch
    load_best_model_at_end=True,     # Save the model that performs best on the validation set
    fp16=True,                       # Enables faster 16-bit precision training (requires GPU)
    report_to="none"
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Calculating the performance metrics

In [9]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
num_emotions=len(emotions)
def compute_metrics(p):
    """
    Computes accuracy, weighted F1, and per-class precision (to check specific emotions).
    """
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted', zero_division=0)
    acc = accuracy_score(p.label_ids, preds)
    class_metrics = precision_recall_fscore_support(p.label_ids, preds, average=None, labels=range(num_emotions), zero_division=0)
    class_precision = class_metrics[0]
    precision_by_class = {emotions[i]: class_precision[i] for i in range(num_emotions)}
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        **{f'precision_{k}': v for k, v in precision_by_class.items()}, # Adds individual precision scores
    }

## Initializing a trainer object from Hugging Face Trainer class

In [10]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
print("Hugging Face Trainer initialized successfully.")

Hugging Face Trainer initialized successfully.


## Training the model

In [11]:
trainer.train()
final_results=trainer.evaluate(test_dataset)
print(final_results)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Precision Fun,Precision Surprise,Precision Neutral,Precision Enthusiasm,Precision Happiness,Precision Hate,Precision Sadness,Precision Empty,Precision Love,Precision Relief,Precision Anger
1,0.349200,0.367162,0.885096,0.888268,0.894924,0.885096,0.922581,0.755082,0.963037,0.998924,0.729825,0.967809,0.678454,0.992661,0.973849,0.978587,0.919028
2,0.282900,0.354562,0.889610,0.891328,0.894542,0.889610,0.894949,0.771513,0.968844,0.974869,0.784091,0.971846,0.708262,0.954545,0.947528,0.963427,0.921986
3,0.213400,0.388078,0.889046,0.890193,0.891950,0.889046,0.887663,0.754302,0.975758,0.960825,0.802927,0.963768,0.735971,0.927242,0.946573,0.949949,0.919355


{'eval_loss': 0.3523584008216858, 'eval_accuracy': 0.8872696502444528, 'eval_f1': 0.889136728171582, 'eval_precision': 0.8925220076432021, 'eval_recall': 0.8872696502444528, 'eval_precision_fun': 0.8941058941058941, 'eval_precision_surprise': 0.7566793893129771, 'eval_precision_neutral': 0.9684106614017769, 'eval_precision_enthusiasm': 0.9741735537190083, 'eval_precision_happiness': 0.7838364167478091, 'eval_precision_hate': 0.9736842105263158, 'eval_precision_sadness': 0.6974935177182369, 'eval_precision_empty': 0.9650959860383944, 'eval_precision_love': 0.9491869918699187, 'eval_precision_relief': 0.9619047619047619, 'eval_precision_anger': 0.9195876288659793, 'eval_runtime': 4.5391, 'eval_samples_per_second': 2343.214, 'eval_steps_per_second': 36.792, 'epoch': 3.0}
